In [3]:
from fastai.basics import *

In [9]:
Adam??

In [11]:
Optimizer??

In [106]:
def update_prev_grad(p, mom, dampening=False, grad_avg=None, **kwargs):
    "Keeps track of the previous gradient, should be one of last cbs. "
    return {'prev_grad': p.grad.data}

In [21]:
average_sqr_grad??

In [113]:
def n_avg_grad(p,lr,nmom=None,n_avg=None,**kwags):
    if n_avg is None: 
        prev_grad=torch.zeros_like(p.grad.data)
        n_avg = p.grad.data-prev_grad
    else:
        n_avg = (1-nmom)*n_avg+nmom*(p.grad.data-prev_grad)
    return {'n_avg': n_avg,'prev_grad':prev_grad}

In [114]:
def n_average_sqr_grad(p,nmom,sqr_mom, prev_grad=None, dampening=True, sqr_avg=None, **kwargs):
    if sqr_avg is None: sqr_avg = torch.zeros_like(p.grad.data)
    damp = 1-sqr_mom if dampening else 1.
    grad = (2-nmom)*p.grad.data+(nmom-1)*prev_grad
    sqr_avg.mul_(sqr_mom).addcmul_(grad,grad, value=damp)
    return {'sqr_avg': sqr_avg}

In [115]:
def Adan(params, lr, mom=0.9, sqr_mom=0.99,nmom=0.9, eps=1e-5, wd=0.01, decouple_wd=True):
    "A `Optimizer` for Adam with `lr`, `mom`, `sqr_mom`, `eps` and `params`"
    cbs = [weight_decay] if decouple_wd else [l2_reg]
    cbs += [partial(average_grad, dampening=True),n_avg_grad, n_average_sqr_grad,adan_step, update_prev_grad]
    return Optimizer(params, cbs, lr=lr,nmom=nmom, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)

In [116]:
l=nn.Linear(4,4)
opt=Adan(l.parameters(),0.01)
print(l.weight)
inp=torch.tensor([.1,.2,.3,.4])
F.mse_loss(l(inp),torch.tensor([1.,2.,3.,4.])).backward()
opt.step()
F.mse_loss(l(inp),torch.tensor([1.,2.,3.,4.])).backward()
opt.step()

Parameter containing:
tensor([[-0.2699,  0.2463,  0.2389,  0.3609],
        [-0.4823,  0.1599, -0.1185,  0.1422],
        [ 0.4541, -0.1880,  0.0794, -0.3760],
        [-0.2717,  0.3392,  0.1453, -0.1475]], requires_grad=True)


UnboundLocalError: local variable 'prev_grad' referenced before assignment

In [73]:
l.weight

Parameter containing:
tensor([[-0.2711, -0.0468,  0.0525, -0.1353],
        [ 0.1062, -0.3647,  0.4919, -0.2543],
        [-0.3699,  0.1654,  0.1101,  0.0434],
        [-0.2083,  0.3848, -0.1086,  0.3713]], requires_grad=True)

In [104]:
def adan_step(p,lr,grad_avg=None,nmom=None,n_avg=None,sqr_avg=None,
             eps=None,**kwargs):
    p.data.addcdiv_(grad_avg+(1-nmom)*n_avg, 
                    (sqr_avg).sqrt() + eps, 
                    value = -lr)

In [99]:
adam_step??

In [ ]:
def Adam(params, lr, mom=0.9, sqr_mom=0.99, eps=1e-5, wd=0.01, decouple_wd=True):
    "A `Optimizer` for Adam with `lr`, `mom`, `sqr_mom`, `eps` and `params`"
    cbs = [weight_decay] if decouple_wd else [l2_reg]
    cbs += [partial(average_grad, dampening=True), average_sqr_grad, step_stat, adam_step]
    return Optimizer(params, cbs, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)